In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def generate_time_series(num_timeseries, n_steps):
    
    freq1, freq2, phase1, phase2 = np.random.rand(4, num_timeseries, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time-phase1) * (freq1 * 10 + 10)) # First sine wave
    series += 0.2 * np.sin((time-phase2) * (freq2 * 20 + 20)) # Second sine wave
    series += 0.1 * (np.random.rand(num_timeseries, n_steps) - 0.5) # Noise

    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

In [4]:
# X_train contains 7000 time series of shape [50, 1], X_train is [7000, 50, 1]
# y_train takes the last element of each series as a label
X_train, y_train = series[:7000, :n_steps ], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

Simple RNN

In [5]:
# input_shape is [None, 1] because a RNN model can process sequences of any length, and this is univariate time series.
# Also we will use the simplest architecture: a single layer with 1 neuron.
simple_rnn = keras.Sequential([
    layers.SimpleRNN(1, input_shape=[None, 1])
])
simple_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = keras.optimizers.Adam(lr=0.01)
simple_rnn.compile(loss="mse", optimizer=optimizer)

In [ ]:
history = simple_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 3s 9ms/step - loss: 0.1289 - val_loss: 0.0270
Epoch 2/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0242 - val_loss: 0.0171
Epoch 3/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0161 - val_loss: 0.0134
Epoch 4/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0130 - val_loss: 0.0119
Epoch 5/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0120 - val_loss: 0.0113
Epoch 6/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0112 - val_loss: 0.0112
Epoch 7/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0114 - val_loss: 0.0112
Epoch 8/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0112 - val_loss: 0.0112
Epoch 9/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0110 - val_loss: 0.0112
Epoch 10/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0113 - val_loss: 0.011